In [252]:
# Leo el fichero y creo 100 0particiones (NO CAMBIA LA PROGRAMACIÓN EL NUMERO DE PARTICIONES!!!!!!!!!) 
# ls = sc.textFile('data/coupon150720.csv',100)
# si no pongo nada spark lo calcula el sólo (por cores normalmente) por ejemplo 3
ls = sc.textFile('data/coupon150720.csv')

In [4]:
# no hago un collect ahora porque puede tardar. IMP no tiene por qué ser las primeras pues está en un cluster 
# ahora en mi PC solo está en mi disco duro y serán las 5 primeras
ls.take(5)

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 u'79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 u'79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 u'79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 u'79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [8]:
# esta si te garantiza que es la primera (util para ver cabeceras)
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [16]:
ls.count()

1232662

In [11]:
ls.cache()
%time ls.count()
%time ls.count()

CPU times: user 8.41 ms, sys: 4.13 ms, total: 12.5 ms
Wall time: 5.43 s
CPU times: user 3.66 ms, sys: 2.34 ms, total: 5.99 ms
Wall time: 4.25 s


1232662

In [12]:
ls.persist()

MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

In [31]:
# Numero de pasajeros de IBERIA
a = ls.map(lambda x: x.split(",")[5])
a.filter(lambda x: x=='IB').take(5)

[u'IB', u'IB', u'IB', u'IB', u'IB']

In [32]:
a.filter(lambda x: x=='IB').count()

26158

In [33]:
# En programación funcional podemos anidar funciones . AHORA EN UNA LINEA
ls.map(lambda x: x.split(",")[5]).filter(lambda x: x=='IB').count()

26158

In [60]:
# $ de IBERIA
int(ls.map(lambda x: x.split(",")[5:7]).filter(lambda x: x[0]=='IB').map(lambda y: float(y[1])).sum())

2828044

In [51]:
def getAerolineaDinero(l):
    elems = l.split(",")
    aerolinea = elems[5]
    dinero = float(elems[6])
    return (aerolinea, dinero)

In [53]:
getAerolineaDinero(ls.first())

(u'9W', 56.79)

In [55]:
# Ya no ponemos lambda porque tenemos definida la funcion(programacion funcional)
dineros = ls.map(getAerolineaDinero)
dineros.take(5)

[(u'9W', 56.79),
 (u'9W', 84.34),
 (u'9W', 60.0),
 (u'9W', 160.63),
 (u'9W', 152.46)]

In [59]:
dinerosIB = dineros.filter(lambda x: x[0] == 'IB')

In [65]:
cantidades = dinerosIB.map(lambda x: x[1])

In [67]:
int(cantidades.sum())

2828044

In [68]:
cantidades.stats()

(count: 26158, mean: 108.113939139, stdev: 168.011485542, max: 5239.93, min: 0.0)

In [72]:
# Calcular la ruta más cara o top ten de las rutas más caras
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [86]:
def getRutaDinero(l):            # para usar en spark sólo un parámetro
    elems = l.split(",")
    origen = elems[2]
    destino = elems[3]
    dinero = float(elems[6])
    ruta = origen + "-" + destino # la usaré más tarde como clave
    return (ruta, dinero)

In [87]:
getRutaDinero(ls.first())

(u'MAA-AUH', 56.79)

In [89]:
rutas = ls.map(getRutaDinero)
rutas.take(3)

[(u'MAA-AUH', 56.79), (u'AUH-CDG', 84.34), (u'CJB-MAA', 60.0)]

In [91]:
rutas.filter(lambda x: x[1] < 7500).sortBy(lambda x: x[1], False).take(5) # primero filtro luego ordeno (más eficiente)

[(u'MAA-AUH', 56.79),
 (u'AUH-CDG', 84.34),
 (u'CJB-MAA', 60.0),
 (u'DEL-DXB', 160.63),
 (u'AUH-IXE', 152.46)]

In [93]:
# quiero quitar repetidos (uso keys no pensar en clave unica de BBDD sino clave para agrupar)
filtrado = rutas.filter(lambda x: x[1] < 7500)
filtrado.cache()

PythonRDD[97] at RDD at PythonRDD.scala:43

In [94]:
filtrado.groupByKey().take(3)

[(u'SZF-IST', <pyspark.resultiterable.ResultIterable at 0x7f621f206c90>),
 (u'DME-RHO', <pyspark.resultiterable.ResultIterable at 0x7f621f206c50>),
 (u'KRK-OSL', <pyspark.resultiterable.ResultIterable at 0x7f621f206bd0>)]

In [98]:
maximos = filtrado.groupByKey().mapValues(lambda vs: max(vs)) # mapvalues porque las claves no intervienen en los calculos

In [100]:
maximos.sortBy(lambda x: x[1], False).take(5)

[(u'BLA-MIA', 7490.33),
 (u'MIA-CCS', 7415.0),
 (u'CCS-MIA', 7415.0),
 (u'RUH-LAX', 7401.94),
 (u'DXB-MAA', 7343.27)]

In [107]:
# La ruta con el precio medio más alto
medias = filtrado.groupByKey().mapValues(lambda vs: sum(vs)/len(vs))
medias.take(5)

[(u'SZF-IST', 77.23333333333332),
 (u'DME-RHO', 120.57857142857144),
 (u'KRK-OSL', 98.0),
 (u'OMS-AAQ', 184.14),
 (u'KBP-IST', 107.67163265306122)]

In [108]:
medias.sortBy(lambda x: x[1], False).take(5)

[(u'BLA-MEC', 5164.355),
 (u'MEC-BLA', 5164.355),
 (u'CCS-MIA', 4350.516296296297),
 (u'MIA-CCS', 4186.95),
 (u'NRT-PPT', 3751.5499999999997)]

In [109]:
# Método más sofisticado de quitar outlayer (hasta ahora <7500) y ahora por categorias (ruta, aerolinea, booking class, etc...)
ls.first() # los campos 2,3,6,8

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [137]:
def getRutaALDIneroClase(l):
    elems = l.split(",")
    ruta = elems[2] + "-" + elems[3]
    al = elems[4]
    dinero = float(elems[6])
    clase = elems[9]
    return ((ruta,al,clase),dinero) # dinero será la clave
getRutaALDIneroClase(ls.first())

((u'MAA-AUH', u'9W', u'H'), 56.79)

In [138]:
import math

In [160]:
# defino un valor limite pro grupo
def valorLimite(vs):
    logs = [math.log(v+1) for v in vs]     # El más 1 es para no error log(0)(no afecta el más 1 porque no cambia la distribucuón a la hora de calcular outlayers)
    media = sum(logs)/len(logs)
    desv = [(x-media)**2 for x in logs]
    var = sum(desv)/len(desv)
    desvStd = math.sqrt(var)
    return math.exp(media+2*desvStd) -1    # Por ejemplo además deshago el log (sólo los altos voy a eliminar)
valorLimite([12,2,3,4,6])

14.346121683250376

In [161]:
rutas = ls.map(getRutaALDIneroClase)
rutas.take(3)

[((u'MAA-AUH', u'9W', u'H'), 56.79),
 ((u'AUH-CDG', u'9W', u'H'), 84.34),
 ((u'CJB-MAA', u'9W', u'H'), 60.0)]

In [162]:
def filtraCuponesEstremos(vs):
    vl = valorLimite(vs)
    f = [v for v in vs if v <= vl]
    return f

In [163]:
grupos = rutas.groupByKey().mapValues(filtraCuponesEstremos)

In [166]:
# Las listas vacías darían error al  alcular la media
def mediaGrupo(vs):
    if (len(vs) == 0):
        return 0.0
    else:
        return sum(vs)/len(vs)

In [167]:
grupos.mapValues(mediaGrupo).sortBy(lambda x: x[1], False).take(5)

[((u'SYD-CGK', u'GA', u'G'), 6355194.0),
 ((u'COO-CDG', u'AF', u'J'), 538599.2),
 ((u'CDG-COO', u'AF', u'J'), 179628.3533333333),
 ((u'DKR-NBO', u'KQ', u'T'), 58253.593333333345),
 ((u'TSN-ICN', u'KE', u'Z'), 38655.45)]

In [170]:
# vamos a filtrar mas, por ejemplo los que tenga >10 cupones porque conjuntos de datos pequeños llevan a error

grupos = rutas.groupByKey()
grupos.take(2)

[((u'TRD-BOO', u'SK', u'V'),
  <pyspark.resultiterable.ResultIterable at 0x7f61f7ce6110>),
 ((u'DPS-TMC', u'GA', u'N'),
  <pyspark.resultiterable.ResultIterable at 0x7f61f7ce6290>)]

In [173]:
gruposfiltrados = grupos.filter(lambda x: len(x[1]) > 10)

In [175]:
r = gruposfiltrados.mapValues(filtraCuponesEstremos).mapValues(mediaGrupo)

In [176]:
r.sortBy(lambda x: x[1], False).take(5)

[((u'MIA-CCS', u'S3', u'S'), 7826.25),
 ((u'CCS-MIA', u'S3', u'B'), 5550.0),
 ((u'CCS-MAD', u'V0', u'Y'), 5418.098666666667),
 ((u'RUH-FRA', u'LH', u'F'), 4792.983),
 ((u'LHR-DOH', u'QR', u'F'), 4275.98923076923)]

In [178]:
# cuantas aeorlineas diferentes hay en el daytaset  - DISTINCT (es muy cara de recursos (incluye ordernar))
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [183]:
ls.map(lambda x: x.split(",")[4]).distinct().count()

359

In [181]:
# son todos los precios en dolares?
ls.map(lambda x: x.split(",")[7]).distinct().count()  # en vez de count si pongo collect abuso de la memoria

1

In [182]:
# Salvar los datos reducidos por ejemplo para tratar con R (un fichero por partición OJO!!!!!)
ls.map(lambda x: x.split(",")[4]).distinct().saveAsTextFile("result.csv")

## Diferencia entre Reduce y Groupbykey (hay que evitar el uso de este último)

In [200]:
# Cálculo de Ingreso total por aerolínea
def getALIngreso(l):
    elems = l.split(",")
    al = elems[4]
    ingreso = float(elems[6])
    return (al,ingreso)
getALIngreso(ls.first())

(u'9W', 56.79)

In [201]:
als = ls.map(getALIngreso)
als.take(3)

[(u'9W', 56.79), (u'9W', 84.34), (u'9W', 60.0)]

In [202]:
totales = als.groupByKey().mapValues(lambda vs: sum(vs))

In [203]:
totales.sortBy(lambda x: x[1],False).take(10)

[(u'GA', 20712226.56999984),
 (u'BA', 12443986.980000038),
 (u'AF', 10221507.909999428),
 (u'QF', 8358587.900000253),
 (u'LH', 7715608.339999713),
 (u'QR', 6935584.369999946),
 (u'SV', 6286299.340001837),
 (u'UA', 5151815.289999981),
 (u'JJ', 5066300.509999913),
 (u'AA', 4151996.4599999674)]

In [204]:
# Ahora usaremos un ReduceByKey (Hay que incluirle una función mapreduce que aplicará a cada grupo)
als.reduceByKey(lambda e,acum: e+acum).take(2)

[(u'', 478.31000000000006), (u'BE', 64073.32999999998)]

In [205]:
als.reduceByKey(lambda e,acum: e+acum).sortBy(lambda x: x[1],False).take(10)

[(u'GA', 20712226.569999855),
 (u'BA', 12443986.980000002),
 (u'AF', 10221507.909999428),
 (u'QF', 8358587.9000001205),
 (u'LH', 7715608.339999718),
 (u'QR', 6935584.369999997),
 (u'SV', 6286299.340001843),
 (u'UA', 5151815.29),
 (u'JJ', 5066300.509999905),
 (u'AA', 4151996.4599999613)]

In [206]:
# comparo
%time als.groupByKey().mapValues(lambda vs: sum(vs)).take(2)
%time als.reduceByKey(lambda e,acum: e+acum).take(2)

CPU times: user 14.5 ms, sys: 16.3 ms, total: 30.8 ms
Wall time: 8.82 s
CPU times: user 9.61 ms, sys: 5.08 ms, total: 14.7 ms
Wall time: 8.04 s


[(u'', 478.31000000000006), (u'BE', 64073.32999999998)]

In [222]:
# Calcular los top 10 aerolineas con mayor media de precio por empleado

# con groupBY es muy sencillo
als.groupByKey().mapValues(lambda vs: sum(vs)/len(vs)).sortBy(lambda x: x[1], False).take(10)

[(u'S3', 5225.068852459017),
 (u'9V', 1488.673986486487),
 (u'GA', 991.2527671691715),
 (u'TN', 964.6472580645161),
 (u'7F', 668.035),
 (u'DT', 579.0006976744188),
 (u'B8', 443.295),
 (u'V0', 438.6122773044147),
 (u'NE', 430.1),
 (u'4M', 377.7557471264368)]

In [247]:
# Con reduce hay dificultad adicional ya que la media no es acumulativa
def calculaMediaReduce(e,acum):
    sumaTotal = acum[0]
    numElems = acum[1]
    return (e[0] + sumaTotal, e[1] + numElems)

In [248]:
# Transformo en tuplas para pasar luego a ReduceByKey en vez de valores 
als.map(lambda x: (x[0],(x[1],1))).take(2)  # x[0] es la clave y (x[1],1) la tupla que apsaremos a la función anterio

[(u'9W', (56.79, 1)), (u'9W', (84.34, 1))]

In [249]:
tr = als.map(lambda x: (x[0],(x[1],1))).reduceByKey(calculaMediaReduce)

In [251]:
tr.mapValues(lambda x: x[0]/x[1]).sortBy(lambda x: x[1], False).take(10)

[(u'S3', 5225.068852459017),
 (u'9V', 1488.6739864864867),
 (u'GA', 991.2527671691723),
 (u'TN', 964.6472580645161),
 (u'7F', 668.035),
 (u'DT', 579.0006976744188),
 (u'B8', 443.295),
 (u'V0', 438.6122773044147),
 (u'NE', 430.1),
 (u'4M', 377.7557471264368)]

### JOINS

In [253]:
def getTCNIngreso(l):
    elems = l.split(",")
    tcn = elems[0]
    ingreso = float(elems[6])
    return (tcn, ingreso)

In [259]:
cs = sc.textFile('data/coupon150720.csv').map(getTCNIngreso)
cs.take(2)

[(u'79062005698500', 56.79), (u'79062005698500', 84.34)]

In [256]:
sc.textFile('data/transm150720.csv').first()

u'79062005698500,TKTT,30,150719,FR,0.0,EUR,T,T,141025,PARA127A8,0.0,EUR,   ,EX,,150719,0.0'

In [257]:
# Haremos el join por el primer número
def getTCNCanal(l):
    elems = l.split(",")
    tcn = elems[0]
    canal = elems[8]
    return (tcn, canal)

In [260]:
ts = sc.textFile('data/transm150720.csv').map(getTCNCanal)
ts.take(2)

[(u'79062005698500', u'T'), (u'79062005924069', u'T')]

In [264]:
# Cuidado un ticket (itineraio compelto) son varios cupones 
ts.cache()
cs.cache()
print(ts.count())
print(cs.count())
# ts.leftOuterJoin(cs).take(3) = cs.leftOuterJoin(ts) porque las claves estan repetidas en cs (el concepto de clave no es igual en SPARK)
ts.leftOuterJoin(cs).count()

631405
1232662


1232662

In [265]:
j = ts.leftOuterJoin(cs).cache()

In [266]:
j.take(2)

[(u'79062005994272', (u'A', 56.27)), (u'79062005994272', (u'A', 56.27))]

In [267]:
j.map(lambda x:x[1]) # me quedo con los valores, las claves no importan en este caso

PythonRDD[581] at RDD at PythonRDD.scala:43

In [268]:
j.map(lambda x: x[1]).reduceByKey(lambda x,y: x+y)

PythonRDD[586] at RDD at PythonRDD.scala:43

In [270]:
j.map(lambda x: x[1]).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], False).take(10)

[(u'A', 99011336.9600026),
 (u'T', 83637221.2200021),
 (u'E', 994305.880000001),
 (u'V', 843207.13),
 (u'', 345827.3100000002)]

In [272]:
# Ejercicio= quiero claves "reales" sin duplicidades
valorTS = cs.groupByKey() # puedo usar Groupbykey porque hay muchas claves
valorTS.take(3)

[(u'79062005558463',
  <pyspark.resultiterable.ResultIterable at 0x7f62038f08d0>),
 (u'79065668432713',
  <pyspark.resultiterable.ResultIterable at 0x7f62038f0810>),
 (u'79062005501193',
  <pyspark.resultiterable.ResultIterable at 0x7f62038f0990>)]

In [274]:
valorTS = valorTS.mapValues(lambda vs: sum(vs))
valorTS.take(3)

[(u'79062005558463', 0.0),
 (u'79065668432713', 286.98),
 (u'79062005501193', 272.96)]

In [276]:
print(ts.count())
print(valorTS.count())

631405
631405


In [279]:
# Ahora tienen el mismo número
j = ts.leftOuterJoin(valorTS)
j.take(3)

[(u'79062005994272', (u'A', 112.54)),
 (u'79065668614400', (u'T', 229.51000000000002)),
 (u'79062005879562', (u'T', 99.50999999999999))]

In [280]:
# ahora no groupByKey pues hay pocas claves y se cargará mucho la memoria
j.map(lambda x: x[1]).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], False).take(5) 

[(u'A', 99011336.96000284),
 (u'T', 83637221.22000207),
 (u'E', 994305.8800000008),
 (u'V', 843207.1299999999),
 (u'', 345827.31)]

In [295]:
# AEROLINEA que ha transportado mayor número de menores

sc.textFile('data/coupon150720.csv').take(5)

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 u'79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 u'79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 u'79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 u'79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [302]:
def getTCNAero(l):
    elems = l.split(",")
    tcn = elems[0]
    aero = elems[5]
    return (tcn, aero)

In [303]:
aes = sc.textFile('data/coupon150720.csv').map(getTCNAero)
aes.take(2)

[(u'79062005698500', u'9W'), (u'79062005698500', u'9W')]

In [282]:
sc.textFile('data/transm150720.csv').take(5)

[u'79062005698500,TKTT,30,150719,FR,0.0,EUR,T,T,141025,PARA127A8,0.0,EUR,   ,EX,,150719,0.0',
 u'79062005924069,TKTT,60,150719,US,0.0,USD,T,T,150719,SJC1S212D,75.38,USD,ADT,CCVI4147,,150719,0.0',
 u'79065668570385,TKTT,30,150719,AE,980.0,AED,T,T,150626,DXBFU3109,190.0,AED,CHD,EX,CA,150719,0.0',
 u'79065668737021,TKTT,0,150719,AE,560.0,AED,T,T,150719,AUHOT3128,640.0,AED,   ,CA,,150719,0.0',
 u'79062006192650,TKTT,60,150719,US,0.0,USD,T,T,150719,SEAEX3160,168.83,USD,ADT,CCVI4563,,150719,0.0']

In [287]:
def getTCNTipo(l):
    elems = l.split(",")
    tcn = elems[0]
    tipo = elems[13]
    return (tcn, tipo)

In [288]:
tis = sc.textFile('data/transm150720.csv').map(getTCNTipo)
tis.take(2)

[(u'79062005698500', u'   '), (u'79062005924069', u'ADT')]

In [305]:
# Es mejor filtrar antes y evito el join con info no deseada
fpax = tis.filter(lambda x: x[1]=='CHD')

In [309]:
j = aes.rightOuterJoin(fpax).map(lambda x: x[1])
j.take(2)   # la clave es la aerolinea

[(u'LH', u'CHD'), (u'OS', u'CHD')]

In [310]:
# Cambio CHD por un 1 para poder calcualr luego en el reduce
j.mapValues(lambda x: 1).reduceByKey(lambda x,y: x+y)

PythonRDD[750] at RDD at PythonRDD.scala:43

In [311]:
j.mapValues(lambda x: 1).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], False).take(5)

[(u'JJ', 5350), (u'SV', 5073), (u'BA', 3556), (u'AF', 3378), (u'AB', 3226)]

In [312]:
# Los cupones más caros que sean de niños
def getTCNIngreso(l):
    elems = l.split(",")
    tcn = elems[0]
    ingreso = float(elems[6])
    return (tcn, ingreso)

In [313]:
cs = sc.textFile('data/coupon150720.csv').map(getTCNIngreso)
cs.take(2)

[(u'79062005698500', 56.79), (u'79062005698500', 84.34)]

In [334]:
fpax.join(cs).map(lambda x: x[1]).sortBy(lambda x: x[1],False).take(5)

[(u'CHD', 14498.76),
 (u'CHD', 13547.31),
 (u'CHD', 9446.75),
 (u'CHD', 7709.74),
 (u'CHD', 6629.51)]

## COGROUP

In [338]:
cs = sc.textFile('data/coupon150720.csv').map(lambda x: x.split(",")[0]).map(lambda x: (x,1))
cs.take(3)

[(u'79062005698500', 1), (u'79062005698500', 1), (u'79062005924069', 1)]

In [340]:
ts = sc.textFile('data/transm150720.csv').map(lambda x: x.split(",")[0]).map(lambda x: (x,1))
ts.take(3)

[(u'79062005698500', 1), (u'79062005924069', 1), (u'79065668570385', 1)]

In [342]:
j = ts.cogroup(cs).cache() # la cacheo y no la tengo que cargar de nuevo luego
j.take(2)

[(u'79062005994272',
  (<pyspark.resultiterable.ResultIterable at 0x7f61f7c2b850>,
   <pyspark.resultiterable.ResultIterable at 0x7f61f7c2bed0>)),
 (u'79065668614400',
  (<pyspark.resultiterable.ResultIterable at 0x7f61f7c2b110>,
   <pyspark.resultiterable.ResultIterable at 0x7f61f7c2b690>))]

In [344]:
j.mapValues(lambda x: (len(x[0]),len(x[1]))).take(3)

[(u'79062005994272', (1, 2)),
 (u'79065668614400', (1, 2)),
 (u'79062005879562', (1, 2))]

In [347]:
# Comprobar que todos son 1 en los tickets
j.mapValues(lambda x: (len(x[0]),len(x[1]))).map(lambda x: x [1][0]).distinct().take(5)

[1]

In [348]:
# Ticket con maximo cupones
j.mapValues(lambda x: (len(x[0]),len(x[1]))).sortBy(lambda x: x[1][1], False).take(5)

[(u'79065669300201', (1, 16)),
 (u'79062005712937', (1, 15)),
 (u'79062005779716', (1, 13)),
 (u'79065668940838', (1, 12)),
 (u'79062005319376', (1, 12))]

In [349]:
# o mejor
j.mapValues(lambda x: len(x[1])).sortBy(lambda x: x[1], False).take(5)

[(u'79065669300201', 16),
 (u'79062005712937', 15),
 (u'79062005779716', 13),
 (u'79065668940838', 12),
 (u'79062005319376', 12)]

### Ejercicio: Histograma con fechas de antelación de compra - vuelo

In [354]:
from time import strptime
from time import mktime
strptime('150904','%y%m%d')
mktime(strptime('150904','%y%m%d'))

1441317600.0

In [360]:
def getFechaVuelo(l):
    elems = l.split(",")
    tcn = elems[0]
    fecha = elems[12]
    return (tcn, fecha)

In [364]:
cs = sc.textFile('data/coupon150720.csv').map(getFechaVuelo).groupByKey().mapValues(lambda x: min(x))
cs.take(3)

[(u'79062005558463', u'150801'),
 (u'79065668432713', u'150823'),
 (u'79062005501193', u'150720')]

In [362]:
def getFechaSalida(l):
    elems = l.split(",")
    tcn = elems[0]
    fecha = elems[9]
    return (tcn, fecha)

In [363]:
ts = sc.textFile('data/transm150720.csv').map(getFechaSalida)
ts.take(3)

[(u'79062005698500', u'141025'),
 (u'79062005924069', u'150719'),
 (u'79065668570385', u'150626')]

In [368]:
j = cs.leftOuterJoin(ts).cache()
j.take(2)

[(u'79062005994272', (u'150720', u'150719')),
 (u'79065668614400', (u'150724', u'150719'))]

In [378]:
def diferenciafechas(x):
    return (mktime(strptime(x[0],'%y%m%d')) - mktime(strptime(x[1],'%y%m%d')))/(3600*24)
j.mapValues(diferenciafechas).take(5)

[(u'79062005994272', 1.0),
 (u'79065668614400', 5.0),
 (u'79062005501193', 1.0),
 (u'79062005552377', 1.0),
 (u'79062005474691', 46.0)]

In [382]:
# mejor con un cogroup
def fechamenor(ds):
    segs=[mktime(strptime(d,'%y%m%d')) for d in ds]
    return min(segs)

In [389]:
# Filtro los nulls
fts = ts.filter(lambda x: x[1] != '')
fcs = cs.filter(lambda x: x[1] != '')

In [390]:
fts.cogroup(fcs).take(2)

[(u'79062005994272',
  (<pyspark.resultiterable.ResultIterable at 0x7f61f7c32dd0>,
   <pyspark.resultiterable.ResultIterable at 0x7f61f7c32910>)),
 (u'79065668614400',
  (<pyspark.resultiterable.ResultIterable at 0x7f61f7c328d0>,
   <pyspark.resultiterable.ResultIterable at 0x7f61f7c32090>))]

In [393]:
r = ts.cogroup(cs).mapValues(lambda x: (fechamenor(x[1])-fechamenor(x[0]))/(3600*24))

In [395]:
r.map(lambda x: (int(x[1]),1)).reduceByKey(lambda x,y: x+y).take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 517.0 failed 1 times, most recent failure: Lost task 0.0 in stage 517.0 (TID 936, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2355, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 2355, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 317, in func
    return f(iterator)
  File "/usr/local/spark/python/pyspark/rdd.py